In [2]:
import requests
import pandas as pd

# API 網址
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate desc&format=JSON'

# 使用 requests 從 API 取得數據
response = requests.get(url)
data = response.json()

# 提取數據欄位並轉換成 pandas DataFrame
# 假設數據在 data['records'] 中
df = pd.DataFrame(data['records'])

# 定義 Excel 文件名稱
excel_filename = 'AQI.xlsx'

# 將 DataFrame 儲存為 Excel 文件
df.to_excel(excel_filename, index=False)

print(f'數據已儲存為 {excel_filename}')

df


數據已儲存為 AQI.xlsx


,sitename,county,aqi,pollutant,status,so2,co,o3,o3_8hr,pm10,...,wind_speed,wind_direc,publishtime,co_8hr,pm2.5_avg,pm10_avg,so2_avg,longitude,latitude,siteid
0,基隆,基隆市,42,,良好,0.9,0.2,33.6,39.5,24,...,1.1,93,2024/04/15 23:00:00,0.2,13,23,1,121.760056,25.129167,1
1,汐止,新北市,63,細懸浮微粒,普通,2.4,0.18,27.3,28.6,25,...,2,32,2024/04/15 23:00:00,0.2,20.5,32,4,121.64081,25.06624,2
2,萬里,新北市,40,,良好,0.7,0.19,38.6,39.2,21,...,2.3,53,2024/04/15 23:00:00,0.2,12.4,23,1,121.689881,25.179667,3
3,新店,新北市,72,細懸浮微粒,普通,0,0.3,21.1,,30,...,1.5,14,2024/04/15 23:00:00,0.3,24.1,35,1,121.537778,24.977222,4
4,土城,新北市,66,細懸浮微粒,普通,1.4,0.43,16.6,23.3,34,...,1,322,2024/04/15 23:00:00,0.4,21.5,35,1,121.451861,24.982528,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,臺南(麻豆),臺南市,46,,良好,0,0.22,18,23.8,26,...,0.2,155,2024/04/15 23:00:00,0.2,14.2,26,0,120.24583056,23.17904722,203
81,屏東(琉球),屏東縣,47,,良好,1.4,0.14,30.5,31.6,22,...,1.3,276,2024/04/15 23:00:00,0.1,14.6,29,1,120.37722,22.35222,204
82,新北(樹林),新北市,72,細懸浮微粒,普通,0.7,0.46,6.7,15.8,32,...,0.1,211,2024/04/15 23:00:00,0.4,24.2,36,0,121.38352778,24.94902778,311
83,大甲（日南國小）,臺中市,51,細懸浮微粒,普通,0.7,0.38,8.5,21.8,28,...,0.4,291,2024/04/15 23:00:00,0.3,15.5,25,0,120.65525382,24.39017512,312


建立超文本

In [3]:
import requests
import pandas as pd
import folium

# API 網址
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate desc&format=JSON'

# 使用 requests 從 API 取得數據
response = requests.get(url)
data = response.json()

# 提取數據欄位並轉換成 pandas DataFrame
df = pd.DataFrame(data['records'])

# 將 AQI 從 string 轉換為數字 (因為 API 可能返回字串)
df['aqi'] = pd.to_numeric(df['aqi'], errors='coerce')

# 建立一個地圖對象，並以台灣的大概中心點作為中心
m = folium.Map(location=[23.5, 121], zoom_start=7)

# 為每個站點添加標記
for index, row in df.iterrows():
    # 根據 AQI 值選擇顏色
    if row['aqi'] <= 50:
        color = 'green'
    elif row['aqi'] <= 100:
        color = 'yellow'
    elif row['aqi'] <= 150:
        color = 'orange'
    elif row['aqi'] <= 200:
        color = 'red'
    else:
        color = 'purple'

    # 確保經度和緯度是有效的
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"{row['sitename']}: AQI={row['aqi']}"
        ).add_to(m)

# 儲存地圖為 HTML 文件
m.save('AirQualityMap.html')
print("地圖已生成並儲存為 AirQualityMap.html")


地圖已生成並儲存為 AirQualityMap.html


儲存為圖片

In [4]:
import requests
import pandas as pd
import folium
from selenium import webdriver
from time import sleep
# 以下用以處理圖片
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep

# API 網址
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate desc&format=JSON'

# 使用 requests 從 API 取得數據
response = requests.get(url)
data = response.json()

# 提取數據欄位並轉換成 pandas DataFrame
df = pd.DataFrame(data['records'])

# 將 AQI 從 string 轉換為數字
df['aqi'] = pd.to_numeric(df['aqi'], errors='coerce')

# 建立一個地圖對象
m = folium.Map(location=[23.5, 121], zoom_start=7)  # 台灣的大概中心點

# 為每個站點添加標記
for index, row in df.iterrows():
    if row['aqi'] <= 50:
        color = 'green'
    elif row['aqi'] <= 100:
        color = 'yellow'
    elif row['aqi'] <= 150:
        color = 'orange'
    elif row['aqi'] <= 200:
        color = 'red'
    else:
        color = 'purple'
    
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"{row['sitename']}: AQI={row['aqi']}"
        ).add_to(m)

# 儲存地圖為 HTML 文件
m.save('AirQualityMap.html')

'''儲存為圖片'''
# 取得當前工作目錄
current_path = os.getcwd()
# HTML 文件的絕對路徑
html_file_path = 'file://' + os.path.join(current_path, 'AirQualityMap.html')

# 建立瀏覽選項
options = Options()
# 無頭模式
options.add_argument('--headless')
# 禁用通知
options.add_argument("--disable-notifications")

# 建立 Chrome WebDriver 實例
driver = webdriver.Chrome(options=options)

try:
    # 載入本地 HTML 文件
    driver.get(html_file_path)
    # 等待一點時間讓文件載入
    sleep(3)

    # 螢幕截圖
    driver.save_screenshot('AirQualityMap.png')
    print("螢幕截圖已儲存圖片 AirQualityMap.png")
finally:
    # 關閉瀏覽器
    driver.quit()





螢幕截圖已儲存圖片 AirQualityMap.png
